In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" #'last' 기본값

# Splitting datasets one feature at a time: decision trees


* 의사 결정 트리 (스무고개)


* Decision Tree Flowchart
    - Decision Block (의사결정 블록, 사각형)
    - Terminal Block (단말 블록, 타원형)
    - Branch (가지)
    
    
* 장점
    - 적은 계산 비용
    - 이해하기 쉬운 학습 결과
    - 누락값 처리
    - 분류와 무관한 특징 처리 가능


* 단점
    - 과적합되기 쉬움 (너무 복잡한 형태)
    
    
* 적용
    - 수치형값, 명목형값

---- 이진트리도 있으나 주로 정렬할때 사용 (여기서는 사용하지 않는다.)<br>
kNN 알고리즘은 데이터에대한 추가적으로 생각할 정보를 주지 않는다. 

_알쓸신잡) <br>
Btreeindex, bitmapindex.. 오라클의 저장단위는 물리적인단위, 논리적인 단위로 나뉘어 져있다. 논리적인 단위가 있는 이유 다른 file system 이여도 저장 가능하도록 하기위해서. NTFS 는 제한없음, FAT32 4G 가 최대 <br>
오라클 데이터 주소값 : 6자리3자리6자리3자리 > 오파블로 (objectId(segmentID), 파일번호(몇번째파일), 데이터블럭ID, RowID)_

## 3.1 Tree construction

** ID3 알고리즘**

* ID3 를 이용한 트리 생성 순서 

    1. 데이터를 가장 잘 나눌 수 있는 틍징을 먼저 찾아서 데이터 집합을 하위 집합으로 분할
        - 정보 이득 (Informaion Gain) 이 가장 큰 특징 (현상태와 그다음상태의 정보 차이) 
        - 엔트로피 (Entropy 정보량) 가 가장 크게 낮아지는 특징
        - 정보량이 많다. = 선택사항이 많다. = 불확실 하다. = 복잡성, 불확정성
    2. 해당 특징을 포함하는 노드 생성
    3. 하위 집합의 모든 데이터가 같은 클래스에 속하면 해당 하위 집합에 대한 분류 종료
    4. 2의 경우가 아니라면 이 하위 집합에 대해 1을 적용
    5. 모든 데이터가 분류될 때까지 (=모든 하위 집합에 대해) 1~4 반복
        - 재귀적방법으로 해결
        - factorial(n) : n x (n-1) x (n-2).... =>  fact(n) = n x fact(n-1)
    
    
노드에 비에 feature 정보가 적을 땐 남아있는 타갯들을 보고 다수결로 해결 


### $ 트리 구조 생성 의사코드 

    데이터 집합에 있는 모든 아이템이 같은 부류항복에 속하는지 확인 :
        if 그렇다면, 분류 항목 표시를 반환
        else 아니면
            데이터를 분할하는 가장 좋은 속성을 찾음
            데이터 집합 분할
            가지 노드 생성
                for 각노드마다 분할 반복
                    create branch 를 호출하고 가지노드에 결과 추가 
            가지 노드를 반환함 


General approach to decision trees

    1. Collect: 모든 방법
    2. Prepare: 명목형값, 연속형값(수지형)은 양자화를 통해 이산형 값으로 변환 
        - 양자화 (Quantizaton) : 끊어지는 값으로 만듬, 이산
    3. Analyze: 모든방법, 트리를 만든 후 시각적 트리 검토
    4. Train: 트리형태로 구조 구성
    5. Test: 학습된 트리로 오류율(error rate) 계산
    6. Use: 모든 지도학습에서 사용 (데이터를 이해하기 위해 사용되기도 함)

    >> Knowledge Representation : 생성된 tree 구조 


* 의사결정트리 알고리즘 종류
    - ID3 (Iterative Dichotomiser 3)
    - C4.5 (successor of ID3)
    - C5.0 (successor of ID4)
    - CART (Classification And Regression Tree)
    - CHAID (CHi-squared Automatic Interaction Detector) : 이 알고리즘은 분류 트리를 계산할 때 다단계 분할을 수행한다.
    - MARS (Multivariate adaptive regression splines) : 더 많은 수치 데이터를 처리하기 위해 결정 트리를 사용한다.
    - 조건부 추론 트리 (Conditional Inference Trees) : 과적합을 피하기 위해 여러 테스트에 대해 보정 분할 기준으로 비 - 파라미터 테스트를 사용하는 통계 기반의 방법이다. 이 방법은 편견 예측 선택 결과와 가지 치기가 필요하지 않다.
    
    https://ko.wikipedia.org/wiki/%EA%B2%B0%EC%A0%95_%ED%8A%B8%EB%A6%AC_%ED%95%99%EC%8A%B5%EB%B2%95
    
    
* 가장 적합한 분할 기준 선택 방법
 - 정보 이득
 - 지니 불순도 
 - 분산 감소 
 
 
* target 정보의 yes 로 판단할지 no 로 판단할지는 양적인 조건으로 판단 quantitative 예) 물고기다. 아니다.  

## 3.1.1 Information gain

* 교제 용어 
    - class : feature  예) 2개의 class (컬럼이 2개)
    - class label : feature 의 실제 value 값 예) yes, no
    - label : feature 의 실제 String 값 예) 2번째 컬럼 값 지느러미여부
    - target : 결과 예) 물고기다. 아니다. 


* 데이터를 분할하기 이전 이후의 정보량(엔트로피) 변화
* 정보 이득이 가장 큰 특징에 대한 분할 수행
* 정보 이득으로 정보의 불확실성(엔트로피) 감소

### 개별 정보량과 엔트로피 

$log_2p(x_i)$

* 개별 정보량 
    * 확률이 낮을 수록 개별 정보량은 커짐 : 엔트로피가 커지는데 기어
        * 로그의 결과에 -1 을 곱한 이유 (0~1 확률은 1/n 분수니까, log(1) 이하의 값은 음수가 나옴) <br>
        확률이 낮을수록 개별 정보량은 커짐 > 엔트로피가 커짐 
    * 정보를 전달(표현) 하는데 몇 자리 2진수(몇비트) 면 충분한가
        * 밑이 2 인 log (컴퓨터로 정보를 전달 하려고 하기 때문에)         
  
  
* 엔트로피 
    * 개별 정보량에 대한 기댓값 <br>
        $H=-\sum_{i=1}^{n}p(x_i)\log_2 P(x_i)$
    * 불확실한 정도, 무질서 정도
    * 확률이 낮은 사건이 많을수록 정보의 엔트로피(불확실성)이 커진다.
        - 로그함수 0 에 가까울수록 -y 값이 커짐
    * 정보의 불확실성이 높다.
        = 어떤 값이 나올지 알기 힘들다.
    * 엔트로피가 높은 원인
        - 모든 사건의 확률이 균등 
            개별정보가 기대값이 가장 많을때, 동전 앞,뒤면 나오는 확률이 같을때<br>
            앞면이 나올 확률이 높은 경우 (예, 8/10) 정보의 불확실 성은 줄어든다. (앞면이 많이 나올테니까) 정보량이 적다. (외부에 남아있는 정보가 적다. 뒷면이 나올 확률 2/10)             
        - 확률이 낮은 사건이 많음 (정보가 다양) 
      


<img src='03.entropy.png' width=500>
--- 엔트로피의 성질 : 집합에 범주가 고르게 분포할 수록 엔트로피 값은 높다. (불확실성 상태가 크다.)
의사결정 나무에서는 엔트로피가 높을 수록 Target 구분을 잘 못해주는 속성 필드가 되며, 낮을 수록 구분을 잘 해주는 유의한 속성 필드가 된다.



* [code] 3.1 calcShannonEnt : 데이터 집합의 새넌 엔트로피 계산 

입력 데이터 집합을 순회하면서 타겟 라벨의 빈도수를 구함

In [2]:
import trees

In [11]:
marinedata, labels = trees.createDataSet()
marinedata, labels

([[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']],
 ['no surfacing', 'flippers'])

In [12]:
# 가능한 모든 분류에 대한 딕셔너리 만들기 
numentries = len(marinedata); numentries
labelcounts = {}

# for 문 
i=0
featvec = marinedata[i]; featvec
current_targetlabel = featvec[-1]; current_targetlabel
# 새로운 키 인 경우 사전에 저장 
if current_targetlabel not in labelcounts.keys():
    labelcounts[current_targetlabel] = 0
    labelcounts
labelcounts[current_targetlabel] += 1
labelcounts

5

[1, 1, 'yes']

'yes'

{'yes': 0}

{'yes': 1}

-- 사전의 get 함수 활용

In [13]:
# 새로운 키 인 경우 사전에 저장 (다른방법) 
# if current_targetlabel not in labelcounts.keys():labelcounts[current_targetlabel] = 0
# labelcounts[current_targetlabel] += 1

labelcounts[current_targetlabel] = labelcounts.get(current_targetlabel, 0) + 1
labelcounts

{'yes': 2}

In [21]:
import math
# 새넌 엔트로피 구하기 
shannonEnt = 0.0
i = 0
# for 문 
key = labelcounts.keys()[i]; key
labelcounts[key]
prob = float(labelcounts[key])/numentries
shannonEnt -= prob * math.log(prob,2) #log base 2
shannonEnt # yes 일때의 확률이 나옴 

'yes'

2

0.5287712379549449

* [code] trees calcShannonEnt 사용 

In [25]:
reload(trees)
marinedata, labels = trees.createDataSet()
shannonEnt = trees.calcShannonEnt(marinedata); shannonEnt

<module 'trees' from 'trees.py'>

0.9709505944546686

shannonEnt 는 target 종류가 늘어날 수록 값이 커진다. <br>
    
    yes 2/5, no 3/5 = shannonEnt 0.9 
        -> yes 1/5, no 3/5, maybe(추가) 1/5 = shannonEnt 1.3 

    정보량이 증가, 복잡도가 높아짐 

## 3.1.2 Splitting the dataset

데이터를 첫 번째 feature 값 (라벨) 로 분할하되 그 값이 '1' 인 생물의 데이터 집합 얻기 <br>
집합을 분할하기 위해서 feature 를 선택해야 한다.     

데이터를 2차원으로 분류 > feature 가 2개 

어떤 feature 를 선택할 것인가


* [code] splitDataSet(dataSet, axis, value) feature 의 value 값에 따라 data 모으기 

In [26]:
retdataset = []
marinedata, labels = trees.createDataSet()

In [40]:
# 첫번째 feature(axis=0) 가 1인 것만 뽑기 
axis=1; value=1

i=0
# for 문 
featvec = marinedata[i]; featvec
# value 가 1 이면 저장하기 해당 feature 값 나머지 데이터 저장 
if featvec[axis] == value:
    # [1,1,'yes'] 에서 [ 만 선택 
    reducedfeatvet = featvec[:axis]; reducedfeatvet
    # [1,1,'yes'] 에서 나머지 1,'yes'] 만 선택 
    # append 면 [, [1,'yes']] 중첩으로 들어감 풀어져 나옴 
    # extend [1, 'yes']
    reducedfeatvet.extend(featvec[axis+1:]); reducedfeatvet
    # append 로 중접으로 쌓음  [[1, 'yes'], [1, 'yes']]
    retdataset.append(reducedfeatvet); retdataset

# axis=1 인 경우 [1, 0, yes] 중에서 1 고르고 yes 를 골라서 붙임 [1, yes]

[1, 1, 'yes']

[1]

[1, 'yes']

[[1, 'yes'], [1, 'yes'], [1, 'yes']]

In [ ]:
# 다른 방법 
# if featvec[axis] == value:
#     reducedfeatvet = featvec[:axis]
#     reducedfeatvet.extend(featvec[axis+1:])
#     retdataset.append(reducedfeatvet)
if featvec[axis] == value:
    del featvec[1]
    retdataset.append(featvec); retdataset

* [code] trees splitDataSet 사용 (모든 feature value 에 대해서 엔트로피 구하기) 

In [37]:
reload(trees)
# 첫번째 피처 값 (라벨)이 1인 데이터 집합의 엔트로피 구하기 
feat01 = trees.splitDataSet(marinedata, 0, 1); feat01
trees.calcShannonEnt(feat01)

<module 'trees' from 'trees.pyc'>

[[1, 'yes'], [1, 'yes'], [0, 'no']]

0.9182958340544896

In [38]:
# 첫번쨰 피처 값이 0인 데이터 집합의 엔트로피 
feat00 = trees.splitDataSet(marinedata, 0, 0); feat00
trees.calcShannonEnt(feat00) # 전부 같은 데이터만 있음 분류할 필요가 없다.

[[1, 'no'], [1, 'no']]

0.0

In [39]:
feat10 = trees.splitDataSet(marinedata, 1, 0); feat10
trees.calcShannonEnt(feat10)
feat11 = trees.splitDataSet(marinedata, 1, 1); feat11
trees.calcShannonEnt(feat11)

[[1, 'no']]

0.0

[[1, 'yes'], [1, 'yes'], [0, 'no'], [0, 'no']]

1.0